In [ ]:
import pathlib
import numpy
import scipy.interpolate
import pandas
import matplotlib
import matplotlib.pyplot as pyplot
import pyvista

In [ ]:
pyvista.set_jupyter_backend("ipyvtklink")
pandas.set_option("display.float_format", lambda x: f"{x:.3e}")
numpy.set_printoptions(precision=15)
%config InlineBackend.figure_formats = ['svg']

In [ ]:
def analytical_soln(r, rp, mu, uavg):
    """Analytical solution to pipe flow.
    
    Arguments
    ---------
    r : float or array-like
        The distance(s) from a point or points to the pipe's center.
    rp : float
        The radius of the target pipe.
    mu : float
        The dynamic viscosity, or the kinematic viscosity if the density is 1.
    uavg : float
        The average flow velocity, or the inlet velocity if it is uniform.
    
    Returns
    -------
    The along-pipe velocity at the point(s).
    """
    rp2 = rp * rp
    return 2 * uavg * (rp2 - r * r) / rp2

In [ ]:
# constants
# ---------
lpipe = 0.14  # m
rpipe = 2.27e-3  # m
mu = 1.48e-5  # m^2 / s
uavg = 0.521646  # m / s
umax = uavg * 2  # m / s
crossarea = numpy.pi * rpipe * rpipe

# cases' parent dir
# -----------------
root = pathlib.Path("/home/pychuang/pipeflow/cases")

# cases
# -----
cases = [16, 32, 64, 128]

In [ ]:
# get data at the last time snapshot for all cases
# ------------------------------------------------

# use pandas to host data for convenience
data = pandas.DataFrame(None, index=pandas.Index(cases, dtype=int, name="Ntheta"))

# readers
data["reader"] = [
    pyvista.get_reader(root.joinpath(f"airflow-pipe-{case}", f"airflow-pipe-{case}.foam"))
    for case in cases
]

# configure readers' behaviors
data["reader"].apply(lambda inp: inp.reader.UpdateTimeStep(1.0))
data["reader"].apply(lambda inp: inp.reader.CreateCellToPointOff())
data["reader"].apply(lambda inp: inp.disable_all_cell_arrays())
data["reader"].apply(lambda inp: inp.enable_cell_array("U"))
data["reader"].apply(lambda inp: inp.disable_all_patch_arrays())
data["reader"].apply(lambda inp: inp.enable_patch_array("internalMesh"))

# read actual data to a new column
data["rawdata"] = data["reader"].apply(lambda inp: inp.read()["internalMesh"])

# get a column for number of cells for convenience
data["ncells"] = data["rawdata"].apply(lambda inp: inp.n_cells)

# get a column for number of each slice layer
data["nxy"] = data.rawdata.apply(
    lambda inp: inp.extract_cells(
        inp.find_cells_within_bounds(
            [-rpipe*1.1, rpipe*1.1, -rpipe*1.1, rpipe*1.1, 0.0, 0.0]
        )
    ).n_cells
)

# get a column for number of layers
data["nz"] = data["ncells"] // data.nxy

# get a column for thickness of each layer
data["dz"] = lpipe / data.nz

In [ ]:
# check the lengths and radius of pipe are correct
# ------------------------------------------------
assert data.rawdata.apply(lambda inp: abs(inp.points[:, 2].max()-lpipe) <= 1e-8).all()
assert data.rawdata.apply(lambda inp: abs(inp.points[:, :2].max()-rpipe) <= 1e-8).all()

In [ ]:
# get the layer of cells that contain the cross-section of z=0.135
# ----------------------------------------------------------------
data["slc0135"] = data.rawdata.apply(
    lambda inp: inp.extract_cells(
        inp.find_cells_within_bounds(
            [-rpipe*1.1, rpipe*1.1, -rpipe*1.1, rpipe*1.1, 0.135, 0.135]
        )
    )
)

# check if we really got one layer of cells
# -----------------------------------------
assert (data.slc0135.apply(lambda inp: inp.n_cells) == data["nxy"]).all()

# change the one-layer 3D mesh to 2D mesh
# ---------------------------------------
data["slc0135"] = data.slc0135.apply(
    lambda inp: inp.slice(
        normal="z", origin=inp.center
    ).compute_cell_sizes(False, True, False)  # also compute the cell areas
)

In [ ]:
# get analytical solution at cell centers at z=0.135
# ---------------------------------------------------

# distances from cell centers to the pipe center
data.slc0135.apply(
    lambda inp: inp.cell_data.set_array(
            numpy.sqrt(numpy.sum(inp.cell_centers(False).points[:, :2]**2, axis=1)),
        "r"
    )
)

# analytical soln
data.slc0135.apply(
    lambda inp: inp.cell_data.set_array(
        analytical_soln(inp["r"], rpipe, mu, uavg),
        "soln"
    )
)

# absolute error
data.slc0135.apply(
    lambda inp: inp.cell_data.set_array(
        numpy.abs(inp["soln"] - inp["U"][:, 2]),
        "abserr"
    )
)

# relative err
data.slc0135.apply(
    lambda inp: inp.cell_data.set_array(
        numpy.abs(inp["abserr"] / inp["U"][:, 2]),
        "relerr"
    )
);

In [ ]:
# get polygon definitions of the cross-sectional cells
# ----------------------------------------------------
data["slcpoly"] = data.slc0135.apply(
    # here we assume all cells are of the same type (i.e., having the same number of vertices)
    lambda inp: inp.points[:, :2][inp.faces.reshape(inp.n_faces, -1)[:, 1:]]
)

In [ ]:
# plot absolute error
# --------------------

# unify colorbar and color range
mapper = matplotlib.cm.ScalarMappable(
    matplotlib.colors.LogNorm(
        vmax=data.slc0135.apply(lambda inp: inp["abserr"].max()).max(),
        vmin=data.slc0135.apply(lambda inp: inp["abserr"].min()).min(),
    ),
    cmap="viridis"
)

# plot
nrows, ncols = 2, 3
fig, axs = pyplot.subplots(nrows, ncols, figsize=(12.5, 7), dpi=192)
for i, case in enumerate(cases):
    poly = matplotlib.collections.PolyCollection(
        data.slcpoly[case],
        facecolor=mapper.to_rgba(data.slc0135.loc[case]["abserr"])
    )
    
    i, j = i // ncols, i % ncols  # note we overwote i here becasue we're lazy
    ax = axs[i, j]
    ax.add_collection(poly)
    ax.set_title(rf"$N_\theta={case}$")
    ax.autoscale_view()
    
    if i == nrows - 1:
        ax.set_xlabel("x (m)")
        ax.set_xticks([-rpipe, 0., rpipe])
    else:
        ax.axes.xaxis.set_visible(False)
    
    if j == 0:
        ax.set_ylabel("y (m)")
        ax.set_yticks([-rpipe, 0., rpipe])
    else:
        ax.axes.yaxis.set_visible(False)

fig.colorbar(mapper, ax=axs, extend="both", aspect=30, label="Absolute error (m)")
fig.suptitle("Absolute error at z=0.135 m")
fig.savefig("abs_err_z_0135.png", dpi=192, facecolor="w", bbox_inches="tight")

In [ ]:
# plot relative error
# --------------------

# unify colorbar and color range
mapper = matplotlib.cm.ScalarMappable(
    matplotlib.colors.LogNorm(
        vmax=data.slc0135.apply(lambda inp: inp["relerr"].max()).max(),
        vmin=data.slc0135.apply(lambda inp: inp["relerr"].min()).min(),
    ),
    cmap="viridis"
)

# plot
nrows, ncols = 2, 3
fig, axs = pyplot.subplots(nrows, ncols, figsize=(12.5, 7), dpi=192)
for i, case in enumerate(cases):
    poly = matplotlib.collections.PolyCollection(
        data.slcpoly[case],
        facecolor=mapper.to_rgba(data.slc0135.loc[case]["relerr"])
    )
    
    i, j = i // ncols, i % ncols  # note we overwote i here becasue we're lazy
    ax = axs[i, j]
    ax.add_collection(poly)
    ax.set_title(rf"$N_\theta={case}$")
    ax.autoscale_view()
    
    if i == nrows - 1:
        ax.set_xlabel("x (m)")
        ax.set_xticks([-rpipe, 0., rpipe])
    else:
        ax.axes.xaxis.set_visible(False)
    
    if j == 0:
        ax.set_ylabel("y (m)")
        ax.set_yticks([-rpipe, 0., rpipe])
    else:
        ax.axes.yaxis.set_visible(False)

fig.colorbar(mapper, ax=axs, extend="both", aspect=30, label="Relative error (m)")
fig.suptitle("Relative error at z=0.135 m")
fig.savefig("rel_err_z_0135.png", dpi=192, facecolor="w", bbox_inches="tight")

In [ ]:
# L-infinity norm in the whole 3D domain
# --------------------------------------
data["Linf"] = data.rawdata.apply(lambda inp: abs((inp["U"][:, 2].max()-umax)/umax))
    
# preparing data for plotting 1st and 2nd order line
x = [data.ncells.iloc[0], data.ncells.iloc[-1]]
y1 = [data.Linf.iloc[0], data.Linf.iloc[0]/(x[1]/x[0])**(1./3)]
y2 = [data.Linf.iloc[0], data.Linf.iloc[0]/(x[1]/x[0])**(2./3)]

# plot
pyplot.figure(figsize=(8, 6), dpi=192, constrained_layout=True)
pyplot.loglog(data.ncells, data.Linf, "k.-", ms=20, lw=3, label="Relative errors")
pyplot.loglog(x, y1, "--", lw=1, label="1st-order convergence")
pyplot.loglog(x, y2, "--", lw=1, label="2nd-order convergence")
pyplot.legend(loc=0)
pyplot.xlabel("Number of cells")
pyplot.ylabel("Relative error")
pyplot.title("Relative error: maximum velocity in the entire domain")
pyplot.savefig("rel_err_linf_entire.png", dpi=192, facecolor="w", bbox_inches="tight")

In [ ]:
# L-infinity norm at cross section z=0.135
# -----------------------------------------
data["Linf0135"] = data.slc0135.apply(lambda inp: abs((inp["U"][:, 2].max()-umax)/umax))
    
# preparing data for plotting 1st and 2nd order line
x = [data.ncells.iloc[0], data.ncells.iloc[-1]]
y1 = [data.Linf0135.iloc[0], data.Linf0135.iloc[0]/(x[1]/x[0])**(1./3)]
y2 = [data.Linf0135.iloc[0], data.Linf0135.iloc[0]/(x[1]/x[0])**(2./3)]

# plot
pyplot.figure(figsize=(8, 6), dpi=192, constrained_layout=True)
pyplot.loglog(data.ncells, data.Linf0135, "k.-", ms=20, lw=3, label="Relative errors")
pyplot.loglog(x, y1, "--", lw=1, label="1st-order convergence")
pyplot.loglog(x, y2, "--", lw=1, label="2nd-order convergence")
pyplot.legend(loc=0)
pyplot.xlabel("Number of cells")
pyplot.ylabel("Relative error")
pyplot.title("Relative error: maximum velocity at z=0.135 m")
pyplot.savefig("rel_err_linf_z_0135.png", dpi=192, facecolor="w", bbox_inches="tight")

In [ ]:
# L2 norm at cross section z=0.135
# --------------------------------
data["Ltwo0135"] = data.slc0135.apply(
    lambda inp: numpy.sqrt(numpy.sum((inp["abserr"]**2) * inp["Area"]))
)
    
# preparing data for plotting 1st and 2nd order line
x = [data.ncells.iloc[0], data.ncells.iloc[-1]]
y1 = [data.Ltwo0135.iloc[0], data.Ltwo0135.iloc[0]/(x[1]/x[0])**(1./3)]
y2 = [data.Ltwo0135.iloc[0], data.Ltwo0135.iloc[0]/(x[1]/x[0])**(2./3)]

# plot
pyplot.figure(figsize=(8, 6), dpi=192, constrained_layout=True)
pyplot.loglog(data.ncells, data.Ltwo0135, "k.-", ms=20, lw=3, label="Relative errors")
pyplot.loglog(x, y1, "--", lw=1, label="1st-order convergence")
pyplot.loglog(x, y2, "--", lw=1, label="2nd-order convergence")
pyplot.legend(loc=0)
pyplot.xlabel("Number of cells")
pyplot.ylabel("Relative error")
pyplot.title("Relative error: velocity L2-norm at z=0.135 m")
pyplot.savefig("rel_err_l2_z_0135.png", dpi=192, facecolor="w", bbox_inches="tight")

In [ ]:
# Area error at cross section z=0.135
# -----------------------------------
data["areaerr"] = data.slc0135.apply(
    lambda inp: abs((numpy.sum(inp["Area"]) - crossarea) / crossarea)
)
    
# preparing data for plotting 1st and 2nd order line
x = [data.ncells.iloc[0], data.ncells.iloc[-1]]
y1 = [data.areaerr.iloc[0], data.areaerr.iloc[0]/(x[1]/x[0])**(1./3)]
y2 = [data.areaerr.iloc[0], data.areaerr.iloc[0]/(x[1]/x[0])**(2./3)]

# plot
pyplot.figure(figsize=(8, 6), dpi=192, constrained_layout=True)
pyplot.loglog(data.ncells, data.areaerr, "k.-", ms=20, lw=3, label="Relative errors")
pyplot.loglog(x, y1, "--", lw=1, label="1st-order convergence")
pyplot.loglog(x, y2, "--", lw=1, label="2nd-order convergence")
pyplot.legend(loc=0)
pyplot.xlabel("Number of cells")
pyplot.ylabel("Relative error")
pyplot.title("Relative error: the area of cross-sections");

# add a reference link regarding the convergence rate of circular area
text = pyplot.annotate(
    "Reference: the area of a circle\n" +
    "https://www.ramsay-maunder.co.uk/downloads/The%20area%20of%20a%20circle.pdf",
    (0.11, 0.12), xycoords='figure fraction',
    url="https://www.ramsay-maunder.co.uk/downloads/The%20area%20of%20a%20circle.pdf",
    bbox={"facecolor": "none", "edgecolor": "k", "pad": 10.0}
)
pyplot.savefig("rel_err_cross_area.png", dpi=192, facecolor="w", bbox_inches="tight")